# Questions
Questions are the corner stone of the Turing SDK. The whole library revolves around defining and operating on question objects. In this notebook we will cover:
1. Building a conceptual understanding of the `ShortAnswerQuestion` object
2. Covering the different methods for defining questions

## Conceptual Understanding of the `ShortAnswerQuestion` object
If you haven't already, please read the `01_rubrics.ipynb` notebook, to gain a comprehensive understanding of the Rubric object. That knowledge will be critical in understanding questions.

The question object is the centeral focus of Turing's SDK. The main goal of the library is to provide a clean interface for creating and defining question objects and then seamlessly grading responses to those questions. Before diving into specifics, it is important to understand the pieces of a question. The question object has three attributes:
- `body` (`str`): The body of the question
- `example_answer` (`str`): The example answer to the question
- `rubric` (`Rubric`): The grading rubic used by the LLM for grading the question

Please see the `01_rubrics.ipynb` for details on defining Rubrics. As you can see, most of the difficulty with implementing questions actually lies in implementing the Rubric itself. Once that is handled, the rest of the question model is pretty straightforward. The `ShortAnswerQuestion` defines a central method, `grade(self, answer: str) -> Tuple[str, float]:` which can be used for grading questions. Every implementation of a Short Answer Question should be motivated with the end goal of calling the `grade` method.

To get to that point, we first need to be able to define questions.

## Different Methods for Defining Questions

The `body` and `example_answer` attributes are simple to define, but definining the `Rubric` can take some more work. After selecting how you want your Rubric to be defined, you then need to choose how that implementation will be integrated with your implementation of the `ShortAnswerQuestion` class. 

We can define a `ShortAnswerQuestion` object in one of three ways. 
1. Without a Rubric (then adding one after instantiation)
2. Directly from a `RubricType` (wrapper for the `Rubric.from_rubric_type` class method)
3. Directly from a data payload (using a dictonary to define the question attributes)

Let's look at each of these examples 

### Without a Rubric

You can create a `ShortAnswerQuestion` without a rubric. In this case, the rubric attribute will be an empty `Rubric` object. We can then use the `add_critiera` method or the `set_rubric` method to define the question's rubric after instantiation. The `add_criteria` method is a wrapper to the `add_criteria` method on the rubric object. Below, we see how these two methods of setting the rubric after instantiation can both be achieved by either adding criteria to the rubric or adding criteria to the question directly.

#### Example

In [ ]:
from turing import ShortAnswerQuestion, Rubric, Objective

# EXAMPLE 1: Defining the question wihtout a rubric, and adding the criteria through the question obect
question = ShortAnswerQuestion(
    question="What is the capital of France?",
    answer="Paris",
)

print(question.rubric.rubric_type) # RubricType.CUSTOM
print(question.rubric.size) # 0 (the size of the default empty rubric)

# Adding the criteria directly to the question
question.add_criteria(Objective.FACTUAL, 1.5)
question.add_criteria(Objective.ANALYSIS, 1.0)
question.add_criteria(Objective.CLARITY, 0.5)

print(question.rubric.rubric_type) # RubricType.CUSTOM
print(question.rubric.size) # 3


# EXAMPLE 2: If you perfer to build the rubric seperately, using the `Rubric` API, you may do so, then add the completed rubric to the question
question = ShortAnswerQuestion(
    question="What is the capital of France?",
    answer="Paris",
)

rubric = Rubric.empty()
rubric.add_criteria(Objective.FACTUAL, 1.5)
rubric.add_criteria(Objective.ANALYSIS, 1.0)
rubric.add_criteria(Objective.CLARITY, 0.5)

question.set_rubric(rubric)

print(question.rubric.rubric_type) # RubricType.CUSTOM
print(question.rubric.size) # 3

# EXAMPLE 3: We can also built the rubric prior to instantiating the question, and then pass the rubric directly to the question as a kwarg
rubric = Rubric.empty()
rubric.add_criteria(Objective.FACTUAL, 1.5)
rubric.add_criteria(Objective.ANALYSIS, 1.0)
rubric.add_criteria(Objective.CLARITY, 0.5)

# Instantiate the question with the rubric object directly
question = ShortAnswerQuestion(
    question="What is the capital of France?",
    answer="Paris",
    rubric=rubric
)

As this example demonstrates, we have plenty of ways to define custom rubrics and set them in our question object using the Turing SDK. These above methods will provide the simplest implementation in terms of additional code needed, as they leverage the built in APIs of the Rubric and Question models to simplify the process.


### Directly from a `RubricType` object

You can also create a `ShortAnswerQuestion` directly from a `RubricType`. The `RubricType` enum class predefines sets of `GradingCriteria` for different use cases. This method of rubric definition is the same as using the `Rubric.from_rubric_type` implementation.

#### Example

In [ ]:
from turing import ShortAnswerQuestion, RubricType

# Example 1: We can define the question and the rubric type, and the rubric will be automatically generated
question = Question.from_rubric_type(
    question="What is the capital of France?",
    answer="Paris",
    rubric_type=RubricType.FACTUAL
)

print(question.rubric.rubric_type) # RubricType.FACTUAL
print(question.rubric.size) # 3


# Example 2: This is the same as using the Rubric API to define the rubric, then passing it to the question
rubric = Rubric.from_rubric_type(RubricType.FACTUAL)
question = ShortAnswerQuestion(
    question="What is the capital of France?",
    answer="Paris",
    rubric=rubric
)

print(question.rubric.rubric_type) # RubricType.FACTUAL
print(question.rubric.size) # 3

As these examples show, we can simply leverage the Rubric's `from_rubric_type` method, through the question object. This creates for less boilerplate code, and a simple, one-line definition for question objects.

### Directly from a Data Payload

Finally, you can create a `ShortAnswerQuestion` directly from a data payload. This involves deserializing a Python object and building the `GradingCriteria` objects directly from the payload. To do this, we will have to override the `from_dict` classmethod. Note, that the `from_dict` method calls the `from_dict` method on the Rubric class under the hood. We can choose to leverage this functionality on the Rubric class.

#### Example

In [ ]:
from turing import ShortAnswerQuestion, Rubric, RubricType

# Custom datapaylod that isnot intrinsically supported by the Turing API
payload = {
    "question": {
        'body': "What is the capital of France?",
        'example_answer': 'Paris',
    }
    'tags': ['geography', 'factual']
}

class CustomQuestion(ShortAnswerQuestion):
    """Custom question object for the Turing API"""

    @staticmethod
    def resolve_rubric_from_tags(tags: List[str]) -> Rubric:
        """Custom method for creating the Rubric based on the question tags."""
        if 'factual' in tags:
            return Rubric.from_rubric_type(RubricType.FACTUAL_RUBRIC)
        if 'analysis' in tags:
            return Rubric.from_rubric_type(RubricType.ANALYTICAL)
        if 'reaserch' in tags:
            return Rubric.from_rubric_type(RubricType.COMPREHENSIVE_RUBRIC)
        else:
            return Rubric.from_rubric_type(RubricType.COMMUNICATION_RUBRIC)

    @classmethod
    def from_dict(cls, data: Dict[str, Union[str, List[str]]]) -> 'CustomQuestion':
        """Override the default `from_dict` method to provide custom parsing logic."""
        # Extract the low-level attributes from the payload
        body = data['question']['body']
        answer = data['question']['example_answer']

        # Extract the tags to be used for the rubric
        tags = data['tags']
        rubric = cls.resolve_rubric_from_tags(tags)
        return cls(
            body=body,
            example_answer=answer,
            rubric=rubric
        )


Similarly, we can further extend these methods to provide a more intricate algorithm for determing RubricTypes. Moreover, we can also use this method to implement custom logic for building custom rubrics from the payload. More on this example can be seen in the `01_rubrics.ipynb` notebook, in the section about defining rubrics from custom payloads.

## Conclusion

As you can see, the question object provides a clean and simple API for creating and grading questions. It seeks to implement itself with the Rubric in such a way that allows developers to ignore or further leverage the Rubric API, based on their specific use case.